<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/gtd_happiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
import pandas as pd
import numpy as np

data2015 = pd.read_csv('/content/drive/My Drive/MENA/2015.csv',encoding = "ISO-8859-1", engine = 'python')
data2016 = pd.read_csv('/content/drive/My Drive/MENA/2016.csv',encoding = "ISO-8859-1", engine = 'python')
data2017 = pd.read_csv('/content/drive/My Drive/MENA/2017.csv',encoding = "ISO-8859-1", engine = 'python')
happiness = data2015.append(data2016.append(data2017))

gtd = pd.read_csv('/content/drive/My Drive/MENA/golbal.csv',encoding = "ISO-8859-1", engine = 'python', usecols=[1,2,3,8,10,19,20,21,22,26,27,29,35,41,71,84,100,101,103,58])
gtd = gtd[(gtd['region_txt'] =='Middle East & North Africa') & (gtd['iyear'] > 2000)]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [0]:
# gtd missing values
numeric_ = gtd._get_numeric_data()
cat = gtd.select_dtypes(include='object')

mean_nan = numeric_.fillna(numeric_.mean()).dropna(axis=1, how='all')
cat_nan = cat.loc[:, cat.isnull().mean() <0.25]
# gtd = pd.merge([mean_nan,cat_nan] , sort=True) 
# for col in gtd.columns:
#     print(col,gtd[col].isna().sum())
gtd = mean_nan.join(cat_nan)


In [0]:
# happiness missing values
numeric_ = happiness._get_numeric_data()
cat = happiness.select_dtypes(include='object')

mean_nan = numeric_.fillna(numeric_.mean()).dropna(axis=1, how='all')
cat_nan = cat.loc[:, cat.isnull().mean() <0.25]
# gtd = pd.merge([mean_nan,cat_nan] , sort=True) 
# for col in gtd.columns:
#     print(col,gtd[col].isna().sum())
happiness = mean_nan.join(cat_nan)